# AWS S3 Operations using Boto3

In this notebook, we will perform several operations on AWS S3 using Boto3. 

These operations include:

- creating an S3 bucket
- writing an object
- listing objects
- reading the contents of an object
- deleting an object
- updating an object
- and handling errors for non-existent objects. 
  
We will also demonstrate how to write files with different prefixes and query files by prefix.

## For this notebook to run...

You will need to have these libraries installed

- `boto3`
- `boto3-stubs[s3]`
- `rich`
- `ipykernel`

The recommended approach is to set up your `pyproject.toml` like so:

```toml
[project]
...
dependencies = ['importlib-metadata; python_version<"3.8"', "boto3"]

[project.optional-dependencies]
stubs = ["boto3-stubs[s3]"]
notebooks = ["jupyterlab", "ipykernel", "rich"]
...
dev = ["cloud-course-project[test,release,static-code-qa,stubs,notebooks]"]
```

So that `pip install --editable './[dev]'` will install all the necessary dependencies into your venv.

## Instructions for setting up autocompletion in Jupyter Notebooks in VS Code

1. Install the development dependencies:
```sh
pip install --editable './[dev]'
```
2. Select the notebook kernel and point it to your virtual environment:
```sh
which python
```
3. Select the VS Code Python interpreter and point it to your virtual environment:
```sh
which python
```
4. Reload the VS Code window (`Ctrl/Cmd + Shift + P` > `Developer: Reload Window`)


In [1]:
# Import necessary libraries
import boto3
from botocore.exceptions import ClientError
from uuid import uuid4  # randomly generated string
from rich import print  # pretty printing
import os
from typing import Optional

try:
    from mypy_boto3_s3 import S3Client
except ImportError:
    print("mypy_boto3_s3 not installed")

In [7]:
import os

# define constants

# Set the profile and region for the AWS SDK (boto3) to use
os.environ["AWS_PROFILE"] = "cloud-course"
os.environ["AWS_REGION"] = "ap-south-1"

# Access the AWS_REGION variable
aws_region = os.environ.get("AWS_REGION", "us-west-2")
print(f"AWS_REGION: {aws_region}")

# Create a session using the specified profile and region
S3_CLIENT: "S3Client" = boto3.client("s3", region_name=aws_region)

BUCKET_NAME = f"cloud-course-bucket-{str(uuid4())[:4]}"

# Single example object
EXAMPLE_OBJECT_KEY = "example/object/file.txt"
EXAMPLE_OBJECT_CONTENT = "This is a test object."

# Multiple example objects
EXAMPLE_OBJECTS = [
    ("example-a/object/file1.txt", "This is a test object."),
    ("example-a/object/file2.txt", "This is another test object."),
    ("example-a/object/file3.txt", "This is yet another test object."),
    ("example-b/object/file1.txt", "This is a test object."),
    ("example-b/object/file2.txt", "This is another test object."),
    ("example-b/object/file3.txt", "This is yet another test object."),
]

print(f"{BUCKET_NAME=}")
print(f"{EXAMPLE_OBJECT_KEY=}")
print(f"{EXAMPLE_OBJECT_CONTENT=}")

AWS_REGION: ap-south-1

BUCKET_NAME='cloud-course-bucket-1a4e'

EXAMPLE_OBJECT_KEY='example/object/file.txt'

EXAMPLE_OBJECT_CONTENT='This is a test object.'

## Create a bucket

Here, we create an S3 bucket. Anything you can do in the AWS console, you can do programatically!

* **

Fix for below error:

```bash
ClientError: An error occurred (IllegalLocationConstraintException) when calling the CreateBucket operation: The unspecified location constraint is incompatible for the region specific endpoint this request was sent to.
```

- [StackOverflow](https://stackoverflow.com/questions/49174673/aws-s3api-create-bucket-bucket-make-exception)

- [AWS repost](https://repost.aws/questions/QUKgr9fG6fSimVSUqK9fvkwQ/illegallocationconstraintexception-error-when-using-s3-create-bucket-code)

- [GitHub Discussion](https://github.com/aws/aws-cli/issues/2603)

- Offical Docs: [S3 / Client / create_bucket](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3/client/create_bucket.html)
  - See under "Request Syntax" section

In [8]:
try:
    from mypy_boto3_s3.type_defs import CreateBucketOutputTypeDef
except ImportError:
    print("boto3-stubs[s3] not installed")


def create_bucket(bucket_name: str) -> Optional["CreateBucketOutputTypeDef"]:
    """
    Create an S3 bucket.

    :param bucket_name: Name of the bucket to create
    :type bucket_name: str
    """
    try:
        response = S3_CLIENT.create_bucket(
            Bucket=bucket_name,
            CreateBucketConfiguration={"LocationConstraint": aws_region},
        )
    except ClientError as e:
        print(f"An error occurred: {e}")
        return None

    return response


# Create the bucket
response = create_bucket(bucket_name=BUCKET_NAME)
print(f"Bucket '{BUCKET_NAME}' created successfully.")
print(response)

Bucket 'cloud-course-bucket-1a4e' created successfully.

{
    'ResponseMetadata': {
        'RequestId': 'V9DH6VJZKTT8DXSX',
        'HostId': 'ElJ0v1+OtY3798VgD4d0ZzdEq9A6kkikpqirKUT4tPovjPrHxualqeUgDYSxpMM6s94b4g9kU5Nio2i5v0rj2w==',
        'HTTPStatusCode': 200,
        'HTTPHeaders': {
            'x-amz-id-2': 
'ElJ0v1+OtY3798VgD4d0ZzdEq9A6kkikpqirKUT4tPovjPrHxualqeUgDYSxpMM6s94b4g9kU5Nio2i5v0rj2w==',
            'x-amz-request-id': 'V9DH6VJZKTT8DXSX',
            'date': 'Fri, 07 Jun 2024 09:04:52 GMT',
            'location': 'http://cloud-course-bucket-1a4e.s3.amazonaws.com/',
            'server': 'AmazonS3',
            'content-length': '0'
        },
        'RetryAttempts': 0
    },
    'Location': 'http://cloud-course-bucket-1a4e.s3.amazonaws.com/'
}

## Write an object to the bucket

In this exercise, you will write an object to the S3 bucket.

"S3 Paths" are URLs of the form `s3://bucket-name/key`, called URI. The `key` is the path to the object in the bucket. For example, `s3://my-bucket-name/my-key` refers to the object with key `my-key` in the bucket `my-bucket-name`.

Some examples:

| Path | Bucket | Object Key |
| --- | --- | --- |
| `s3://my-bucket/images/profile.jpeg` | `my-bucket` | `images/profile.jpeg` |
| `s3://my-bucket/data/2021/01/01/data.csv` | `my-bucket` | `data/2021/01/01/data.csv` |
| `s3://my-bucket/file.json` | `my-bucket` | `file.json` |


In [9]:
try:
    from mypy_boto3_s3.type_defs import PutObjectOutputTypeDef
except ImportError:
    print("boto3-stubs[s3] not installed")


def write_text_object_to_s3(
    bucket_name: str,
    object_key: str,
    object_content: str,
) -> Optional["PutObjectOutputTypeDef"]:
    """
    Write an object to an S3 bucket.

    :param bucket_name: Name of the bucket to write to
    :param object_key: Key of the object to write
    :param object_content: Content of the object to write
    :return: Response from the put_object call
    """
    response = S3_CLIENT.put_object(Bucket=bucket_name, Key=object_key, Body=object_content)
    return response


# Write the single example to S3
response = write_text_object_to_s3(
    bucket_name=BUCKET_NAME,
    object_key=EXAMPLE_OBJECT_KEY,
    object_content=EXAMPLE_OBJECT_CONTENT,
)
print(response)

# Write the rest of the examples to S3
for object_key, object_content in EXAMPLE_OBJECTS:
    print(f"Writing object to path 's3://{BUCKET_NAME}/{object_key}'")
    write_text_object_to_s3(
        bucket_name=BUCKET_NAME,
        object_key=object_key,
        object_content=object_content,
    )

{
    'ResponseMetadata': {
        'RequestId': 'MMAAFZVBER156GKX',
        'HostId': 'NGofBskUm0GiFK7tJ/WLxTWSoFkeOwVLgBVHYUSzI4NqLhm/8M6OzBzt1iUIF+/XnsNGgOp5ifE=',
        'HTTPStatusCode': 200,
        'HTTPHeaders': {
            'x-amz-id-2': 'NGofBskUm0GiFK7tJ/WLxTWSoFkeOwVLgBVHYUSzI4NqLhm/8M6OzBzt1iUIF+/XnsNGgOp5ifE=',
            'x-amz-request-id': 'MMAAFZVBER156GKX',
            'date': 'Fri, 07 Jun 2024 09:07:22 GMT',
            'x-amz-server-side-encryption': 'AES256',
            'etag': '"fac2cbcd7b7417c0325922b689019c65"',
            'server': 'AmazonS3',
            'content-length': '0'
        },
        'RetryAttempts': 0
    },
    'ETag': '"fac2cbcd7b7417c0325922b689019c65"',
    'ServerSideEncryption': 'AES256'
}

Writing object to path 's3://cloud-course-bucket-1a4e/example-a/object/file1.txt'

Writing object to path 's3://cloud-course-bucket-1a4e/example-a/object/file2.txt'

Writing object to path 's3://cloud-course-bucket-1a4e/example-a/object/file3.txt'

Writing object to path 's3://cloud-course-bucket-1a4e/example-b/object/file1.txt'

Writing object to path 's3://cloud-course-bucket-1a4e/example-b/object/file2.txt'

Writing object to path 's3://cloud-course-bucket-1a4e/example-b/object/file3.txt'

## Read the content of an object

In this exercise, you will read the content of an object from the S3 bucket.

In [10]:
from botocore.response import StreamingBody


# S3 is a blob store, so we need to read the content of the object as a bytestream
# and then interpret the bytes as text. This function reads the content of an object


def read_text_object_from_s3(bucket_name: str, object_key: str, verbose: bool = False) -> str | None:
    """
    Read the content of an object from an S3 bucket.

    :param bucket_name: Name of the bucket to read from
    :param object_key: Key of the object to read
    :return: Content of the object
    """
    response = S3_CLIENT.get_object(Bucket=bucket_name, Key=object_key)

    # Note, we need to read the bytestream from the blob and choose how we wish
    # to interpret the bytes. In this case, we interpret them as utf-8 encoded text.
    content_streaming_body: StreamingBody = response["Body"]
    content: str = content_streaming_body.read().decode("utf-8")
    return content


# Read the content of the example object
content = read_text_object_from_s3(BUCKET_NAME, EXAMPLE_OBJECT_KEY)
if content:
    print(f"Content of object '{EXAMPLE_OBJECT_KEY}':\n'{content}'")

Content of object 'example/object/file.txt':
'This is a test object.'

In [ ]:
# S3 is a blob storage, which means that S3 is key-value object/blob storage service.
# Meaning that keys maps to big blobs(bytes) of data.

# And since it is a key-value store, that is why folders are not a thing in S3. They are just prefixes in the keys.
# For example, if you have a key like `folder1/folder2/file.txt`, then `folder1` and `folder2` are just prefixes in the key.

# For example, if you want to rename a folder in S3, you would have to iterate over every single object that looks like a folder
# and move that object from it's original location to the new location.

# And if you want to delete a folder in S3,
# you would have to copy all the objects in that folder to a new folder and then delete the old folder.

## Exercise #1 - List objects in the bucket

What if there were more than 1,000 objects in the bucket? How would you list all of them?

Hints
- Look into ["boto3 paginators"](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/paginators.html#creating-paginators)
- OR Look into the [boto3 "resource API"](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/resources.html#resources), a powerful, but potentially less performant, object-oriented way
  of interacting with AWS resources via boto3
- OR consider using "Continuation Tokens" with the `list_objects_v2` method.


* **

- [Creating paginators](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/paginators.html#creating-paginators)

- [Paginators boto3-stubs](https://youtype.github.io/boto3_stubs_docs/mypy_boto3_s3/paginators/#listdirectorybucketspaginator)

- Difference b/w [`S3.Client.list_objects`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3/client/list_objects.html) and [`S3.Client.list_objects_v2`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3/client/list_objects_v2.html):

  - [StackOverflow](https://stackoverflow.com/questions/37534077/what-is-the-difference-between-boto3-list-objects-and-list-objects-v2)

In [35]:
try:
    from mypy_boto3_s3.type_defs import ListObjectsV2OutputTypeDef, ListObjectsOutputTypeDef
    from mypy_boto3_s3.paginator import _PageIterator, ListObjectsV2Paginator
except ImportError:
    print("boto3-stubs[s3] not installed")


def list_all_object_keys_in_bucket(bucket_name: str) -> list[str]:
    """
    List all objects in an S3 bucket.

    :param bucket_name: Name of the bucket to list objects from
    :return: List of object keys
    """
    # respone: ListObjectsOutputTypeDef = S3_CLIENT.list_objects(Bucket=bucket_name)
    # return respone

    # Create a reusable Paginator
    paginator: ListObjectsV2Paginator = S3_CLIENT.get_paginator("list_objects_v2")

    # Create a PageIterator from the Paginator
    page_iterator: _PageIterator[ListObjectsV2OutputTypeDef] = paginator.paginate(Bucket=bucket_name)

    object_keys: list[str] = []

    for page in page_iterator:
        for content in page.get("Contents", []):
            object_keys.append(content.get("Key"))

    return object_keys

In [13]:
# resp = list_all_object_keys_in_bucket(BUCKET_NAME)
# resp.keys()
# resp["Contents"]
# resp["ResponseMetadata"]

In [34]:
# List all objects in the bucket
objects = list_all_object_keys_in_bucket(BUCKET_NAME)
if objects:
    print("Objects in bucket:")
    for obj in objects:
        print(f" - {obj}")

Objects in bucket:

- example-a/object/file1.txt

- example-a/object/file2.txt

- example-a/object/file3.txt

- example-b/object/file1.txt

- example-b/object/file2.txt

- example-b/object/file3.txt

- example/object/file.txt

## Exercise #2 - Update the object (overwrite with new content)

In this exercise, you will update the content of an existing object in the S3 bucket.

In [36]:
new_content = "This is updated content."

previous_content = read_text_object_from_s3(BUCKET_NAME, EXAMPLE_OBJECT_KEY)
print(f"Previous content of object '{EXAMPLE_OBJECT_KEY}':\n{previous_content}")

# the "PUT object" command is an upsert, so it will overwrite existing files
response = write_text_object_to_s3(BUCKET_NAME, EXAMPLE_OBJECT_KEY, new_content)
print(response)

# Read the updated content of the object
updated_content = read_text_object_from_s3(BUCKET_NAME, EXAMPLE_OBJECT_KEY)
if updated_content:
    print(f"Content of updated object '{EXAMPLE_OBJECT_KEY}':\n{updated_content}")

Previous content of object 'example/object/file.txt':
This is a test object.

{
    'ResponseMetadata': {
        'RequestId': 'PDTJRWEK3R7CBS8E',
        'HostId': 'sj6tzNydFzl15YYqDYaYWLLcvO3Wh9C1CRfyRAg2CgzSWq/D5GpVjEoeOpQLXzCws8Qrt2kXO5s=',
        'HTTPStatusCode': 200,
        'HTTPHeaders': {
            'x-amz-id-2': 'sj6tzNydFzl15YYqDYaYWLLcvO3Wh9C1CRfyRAg2CgzSWq/D5GpVjEoeOpQLXzCws8Qrt2kXO5s=',
            'x-amz-request-id': 'PDTJRWEK3R7CBS8E',
            'date': 'Fri, 07 Jun 2024 10:03:15 GMT',
            'x-amz-server-side-encryption': 'AES256',
            'etag': '"c5a01ee632fac2f88da9e0aa5a3d8cdd"',
            'server': 'AmazonS3',
            'content-length': '0'
        },
        'RetryAttempts': 0
    },
    'ETag': '"c5a01ee632fac2f88da9e0aa5a3d8cdd"',
    'ServerSideEncryption': 'AES256'
}

Content of updated object 'example/object/file.txt':
This is updated content.

## Exercise #3 - Delete the object from the bucket

In this exercise, you will delete an object from the S3 bucket.

In [38]:
try:
    from mypy_boto3_s3.type_defs import DeleteObjectOutputTypeDef
except ImportError:
    print("boto3-stubs[s3] not installed")


def delete_object_from_s3(
    bucket_name: str,
    object_key: str,
) -> Optional["DeleteObjectOutputTypeDef"]:
    """
    Delete an object from an S3 bucket.

    :param bucket_name: Name of the bucket to delete the object from
    :param object_key: Key of the object to delete
    :return: Response from the delete_object call
    """
    # fill this out...
    response = S3_CLIENT.delete_object(Bucket=bucket_name, Key=object_key)
    return response


# Delete the example object
response = delete_object_from_s3(bucket_name=BUCKET_NAME, object_key=EXAMPLE_OBJECT_KEY)
print(response)

{
    'ResponseMetadata': {
        'RequestId': 'T951VFT0H5D17NPD',
        'HostId': 'WcgG/dqToctle/61eanlxxL85qJBGHCXkBLewbXD2xhyjZMKdU8cs+glzK8vyTnaD1mtZVWGOy5EHAmILrJ+yw==',
        'HTTPStatusCode': 204,
        'HTTPHeaders': {
            'x-amz-id-2': 
'WcgG/dqToctle/61eanlxxL85qJBGHCXkBLewbXD2xhyjZMKdU8cs+glzK8vyTnaD1mtZVWGOy5EHAmILrJ+yw==',
            'x-amz-request-id': 'T951VFT0H5D17NPD',
            'date': 'Fri, 07 Jun 2024 10:08:57 GMT',
            'server': 'AmazonS3'
        },
        'RetryAttempts': 0
    }
}

## Exercise #4 - Try to read a non-existent object

In this exercise, you will attempt to read a non-existent object from the S3 bucket. 

Catch, suppress, and print the `ClientError` exception that is raised when you try to read a non-existent object.

In [40]:
print(f"Trying to read the deleted object at 's3://{BUCKET_NAME}/{EXAMPLE_OBJECT_KEY}' ...")

delete_object_from_s3(bucket_name=BUCKET_NAME, object_key=EXAMPLE_OBJECT_KEY)

try:
    # try to read the deleted object
    read_text_object_from_s3(BUCKET_NAME, EXAMPLE_OBJECT_KEY)
except ClientError as err:
    print("Key not found")
    assert "NoSuchKey" in str(err)

Trying to read the deleted object at 's3://cloud-course-bucket-1a4e/example/object/file.txt' ...

Key not found

## Exercise #5 - Try to delete a non-existent object

**Note:** the result of calling `s3_client.delete_object` on a non-existent object is not what you might expect. It succeeds whether or not there exists an object with the given key.

**Note:** The HTTP status code `204` means `No Content`. Or in other words, the request is successful
but there was nothing to delete.

In [41]:
non_existant_object_key = EXAMPLE_OBJECT_KEY + "_non_existent"

# delete the non-existent object
# ...
response = delete_object_from_s3(bucket_name=BUCKET_NAME, object_key=non_existant_object_key)

print(response)

{
    'ResponseMetadata': {
        'RequestId': '8W1NVJ8EAQJ6PQN1',
        'HostId': 'uYfMnRvDhftGZc/ZPDFuBEAHOQ4JJzZX84LQ2pqY1hTHtB2MssFj0Iu713vNrSlMFEyNuW6PpTE=',
        'HTTPStatusCode': 204,
        'HTTPHeaders': {
            'x-amz-id-2': 'uYfMnRvDhftGZc/ZPDFuBEAHOQ4JJzZX84LQ2pqY1hTHtB2MssFj0Iu713vNrSlMFEyNuW6PpTE=',
            'x-amz-request-id': '8W1NVJ8EAQJ6PQN1',
            'date': 'Fri, 07 Jun 2024 10:11:49 GMT',
            'server': 'AmazonS3'
        },
        'RetryAttempts': 0
    }
}

## Exercise #6 - Error handling when deleting an object

In this exercise, you will implement error handling to raise a `FileNotFoundError` if you try to delete a non-existent object, i.e., one that has already been deleted or was never written.

Hint, the [`s3_client.head_object(...)`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3/client/head_object.html) method raises an error with status code `404 - File Not Found`
if no file exists for the given object key.

In [47]:
# S3_CLIENT.head_object(Bucket=BUCKET_NAME, Key=non_existant_object_key)

In [45]:
HTTP_FILE_NOT_FOUND_ERROR_CODE = "404"


class S3FileNotFoundError(Exception):
    """Raise this exception when an object at a given path is not found in S3."""

    def __init__(self, bucket_name: str, object_key: str) -> None:
        self.bucket_name = bucket_name
        self.object_key = object_key
        self.message = f"Object not found at 's3://{bucket_name}/{object_key}'"

    def __str__(self) -> str:
        return self.message


def delete_object_or_error_if_not_exists(bucket_name: str, object_key: str) -> None:
    """
    Delete an object from an S3 bucket with error handling for non-existent objects.

    :param bucket_name: Name of the bucket to delete the object from
    :param object_key: Key of the object to delete

    :raises S3FileNotFoundError: if no object exists at the given path
    :raises ClientError: if an unexpected error occurs when using S3 that is not due to file not found
    """
    # fill this out...
    try:
        S3_CLIENT.head_object(Bucket=bucket_name, Key=object_key)
        response = S3_CLIENT.delete_object(Bucket=bucket_name, Key=object_key)
        return response
    except ClientError as err:
        # If a 404 error is thrown, the object does not exist
        err_is_due_to_file_not_found = err.response["Error"]["Code"] == HTTP_FILE_NOT_FOUND_ERROR_CODE
        if err_is_due_to_file_not_found:
            raise S3FileNotFoundError(bucket_name, object_key)
        else:
            # Re-raise the exception if it's a different error
            raise


# Try to delete the non-existent example object with error handling
try:
    delete_object_or_error_if_not_exists(BUCKET_NAME, EXAMPLE_OBJECT_KEY)
except S3FileNotFoundError as err:
    print(err)

Object not found at 's3://cloud-course-bucket-1a4e/example/object/file.txt'

## Exercise #7 - List objects in the bucket to confirm they were deleted in the previous exercises

In this exercise, you will list all objects in the S3 bucket to confirm that the object has been deleted.

In [48]:
# List all objects in the bucket
objects = list_all_object_keys_in_bucket(BUCKET_NAME)

if objects:
    print("Objects in bucket:")
    for obj in objects:
        print(f" - {obj}")
else:
    print("Bucket is empty.")

Objects in bucket:

- example-a/object/file1.txt

- example-a/object/file2.txt

- example-a/object/file3.txt

- example-b/object/file1.txt

- example-b/object/file2.txt

- example-b/object/file3.txt

## Exercise 8 - List objects by prefix

In this exercise, you will list objects in the S3 bucket by prefix.

In [52]:
# S3_CLIENT.list_objects_v2(Bucket=BUCKET_NAME, Prefix="example-a/")['Contents']

In [51]:
def list_all_objects_in_bucket_by_prefix(bucket_name: str, prefix: str) -> list[str]:
    """
    List objects in an S3 bucket by prefix.

    :param bucket_name: Name of the bucket to list objects from
    :param prefix: Prefix to filter objects by
    :return: List of object keys
    """
    # fill this out...
    response = S3_CLIENT.list_objects_v2(Bucket=BUCKET_NAME, Prefix=prefix)
    return [content.get("Key") for content in response.get("Contents", [])]


# List objects by prefix
prefix = "example-a/"
objects_by_prefix = list_all_objects_in_bucket_by_prefix(BUCKET_NAME, prefix)

if objects_by_prefix:
    print(f"Objects with prefix '{prefix}':")
    for obj in objects_by_prefix:
        print(f" - {obj}")
else:
    print(f"No objects found with prefix '{prefix}'.")

Objects with prefix 'example-a/':

- example-a/object/file1.txt

- example-a/object/file2.txt

- example-a/object/file3.txt

## Exercise 9 - Delete a bucket, no matter what!

In this exercise, you will delete the S3 bucket. Your bucket may have objects in it. Does that matter?

- **Ans:**
  AWS does not allow you to delete a bucket that has objects in it. You must first delete all objects in the bucket before you can delete the bucket itself.


Write a function to delete your bucket. 

***Be careful to point it at the right bucket using the right
AWS credentials--or you might delete the wrong bucket!***

In [53]:
list_all_object_keys_in_bucket(BUCKET_NAME)

['example-a/object/file1.txt',
 'example-a/object/file2.txt',
 'example-a/object/file3.txt',
 'example-b/object/file1.txt',
 'example-b/object/file2.txt',
 'example-b/object/file3.txt']

In [118]:
import re
import boto3
from botocore.exceptions import ClientError

try:
    from mypy_boto3_s3.type_defs import EmptyResponseMetadataTypeDef
except ImportError:
    print("boto3-stubs[s3] not installed")


def delete_bucket(bucket_name: str) -> Optional["EmptyResponseMetadataTypeDef"]:
    """
    Delete an S3 bucket, including all its objects.

    If the bucket does not exist, no error is raised.

    :param bucket_name: Name of the bucket to delete
    :return: Response from the delete_bucket call or None if there is no bucket.
    """

    try:
        object_keys = list_all_object_keys_in_bucket(bucket_name)
    except ClientError as err:
        if "NoSuchBucket" in str(err):
            return
        raise

    if object_keys:
        delete_objects_format = {"Objects": [{"Key": obj} for obj in object_keys]}
        S3_CLIENT.delete_objects(Bucket=bucket_name, Delete=delete_objects_format)
        print(f"Deleted objects in bucket '{bucket_name}'. Proceeding to delete the bucket.")
    else:
        print(f"Bucket '{bucket_name}' is empty. Proceeding to delete the bucket.")

    try:
        response: "EmptyResponseMetadataTypeDef" = S3_CLIENT.delete_bucket(Bucket=bucket_name)
        print(f"Bucket '{bucket_name}' deleted successfully!")
        return response
    except ClientError as err:
        if "NoSuchBucket" in str(err):
            return
        raise


response = delete_bucket(BUCKET_NAME)
print(response)

Deleted objects in bucket 'cloud-course-bucket-1a4e'. Proceeding to delete the bucket.

Bucket 'cloud-course-bucket-1a4e' deleted successfully!

{
    'ResponseMetadata': {
        'RequestId': 'KNV84Q6ABDQNQV8H',
        'HostId': 'rAUDOXf9z3/djqkUu1UqW0HqFxWsviMHF1zlNsaBT2Q9DJ81r7zYTtftg4TpUR78h3MT9SS+9AE=',
        'HTTPStatusCode': 204,
        'HTTPHeaders': {
            'x-amz-id-2': 'rAUDOXf9z3/djqkUu1UqW0HqFxWsviMHF1zlNsaBT2Q9DJ81r7zYTtftg4TpUR78h3MT9SS+9AE=',
            'x-amz-request-id': 'KNV84Q6ABDQNQV8H',
            'date': 'Fri, 07 Jun 2024 11:57:38 GMT',
            'server': 'AmazonS3'
        },
        'RetryAttempts': 0
    }
}

In [64]:
response = delete_bucket(BUCKET_NAME)
print(response)

Bucket deletion failed/Bucket not exists: cloud-course-bucket-1a4e

Error: An error occurred (NoSuchBucket) when calling the ListObjectsV2 operation: The specified bucket does not 
exist

None

## (Optional) Bonus Exercise #1 - Recursively upload a local directory to S3

In [108]:
from pathlib import Path

# CWD = Path.cwd().resolve()


# create a test dir locally with sample files
# test_dir = CWD / "test_dir"
test_dir = Path("test_dir")
test_dir.mkdir(parents=True, exist_ok=True)
(test_dir / "file1.txt").write_text("This is file 1.")
(test_dir / "file2.txt").write_text("This is file 2.")
(test_dir / "subdir").mkdir(parents=True, exist_ok=True)
(test_dir / "subdir" / "file3.txt").write_text("This is file 3.")

15

In [109]:
# # # Recursively list all files and subdirectories
# for path in test_dir.rglob('*'):
#     if path.is_file():
#         print(f"File: {path}")
#     elif path.is_dir():
#         print(f"Directory: {path}")

# Recursively list all files and subdirectories with relative paths
for path in test_dir.rglob("*"):
    relative_path = path.relative_to(test_dir)
    if path.is_file():
        # print(relative_path.name)
        print(f"File: {relative_path}")
        with path.open() as f:
            content = f.read()
            print(f"Content: {content}")
    # elif path.is_dir():
    #     print(f"Directory: {relative_path}")

File: file1.txt

Content: This is file 1.

File: file2.txt

Content: This is file 2.

File: subdir/file3.txt

Content: This is file 3.

In [120]:
from pathlib import Path
from typing import Generator


def recursively_upload_dir_to_bucket(
    local_dir_fpath: str | Path,
    bucket_name: str,
    target_root_prefix_in_bucket: str = "",
):
    """
    Recurse through a local directory and upload all files to S3 under a target prefix.

    The object keys within the bucket should be the relative paths of the files within the local directory.

    Example:

    path/to/local_dir/
    ├── file1.txt
    ├── file2.txt
    └── subdir
        └── file3.txt

    Would be uploaded to

    s3://bucket-name/target_root_prefix_in_bucket/
    ├── file1.txt
    ├── file2.txt
    └── subdir/
        └── file3.txt
    """
    for path in local_dir_fpath.rglob("*"):
        relative_path = path.relative_to(local_dir_fpath)
        if path.is_file():
            # upload the file to S3
            with path.open("r") as f:
                content = f.read()

            if target_root_prefix_in_bucket:
                # object_key = f"{target_root_prefix_in_bucket}/{str(relative_path)}"
                object_key = target_root_prefix_in_bucket + str(relative_path)
            else:
                object_key = str(relative_path)

            if content:
                print(f"Writing object to path 's3://{bucket_name}/{object_key}'")
                write_text_object_to_s3(
                    bucket_name=bucket_name,
                    object_key=object_key,
                    object_content=content,
                )

    print(
        f"Directory '{local_dir_fpath}' uploaded successfully to S3 bucket '{bucket_name}' with prefix '{target_root_prefix_in_bucket}'."
    )
    return

In [119]:
from pathlib import Path
from typing import Generator


# create a test dir locally with sample files
test_dir = Path("test_dir")
test_dir.mkdir(parents=True, exist_ok=True)
(test_dir / "file1.txt").write_text("This is file 1.")
(test_dir / "file2.txt").write_text("This is file 2.")
(test_dir / "subdir").mkdir(parents=True, exist_ok=True)
(test_dir / "subdir" / "file3.txt").write_text("This is file 3.")

# clean up the bucket
delete_bucket(BUCKET_NAME)
create_bucket(bucket_name=BUCKET_NAME)

recursively_upload_dir_to_bucket(
    bucket_name=BUCKET_NAME,
    local_dir_fpath=test_dir,
    target_root_prefix_in_bucket="test-root-dir/",
)

# List all objects in the bucket at the test target root
objects = list_all_objects_in_bucket_by_prefix(
    bucket_name=BUCKET_NAME,
    prefix="test-root-dir/",
)
print(objects)

Writing object to path 's3://cloud-course-bucket-1a4e/test-root-dir/file1.txt'

Writing object to path 's3://cloud-course-bucket-1a4e/test-root-dir/file2.txt'

Writing object to path 's3://cloud-course-bucket-1a4e/test-root-dir/subdir/file3.txt'

Directory 'test_dir' uploaded successfully to S3 bucket 'cloud-course-bucket-1a4e' with prefix 'test-root-dir'.

['test-root-dir/file1.txt', 'test-root-dir/file2.txt', 'test-root-dir/subdir/file3.txt']

## (Optional) Bonus Exercise #2 - Rename a "folder" in an S3 bucket

S3 is a key-value store for blobs of bytes. There is no way to "rename" a "folder" in S3
without changing the key names of each object in the "folder".

To really feel the weight and implications of this fact, go through this exercise 🤣

Ultimately, you have to copy each object one at a time, and delete the old object. For large buckets, e.g.
data lakes with millions of files, this is a slow process.

In [124]:
def rename_folder_in_bucket(
    bucket_name: str,
    old_folder_prefix: str,
    new_folder_prefix: str,
):
    """
    Rename a "folder" in an S3 bucket.

    Example:

    Given the following structure in S3:

    s3://bucket-name/<old_folder_prefix>/
    ├── file1.txt
    ├── file2.txt
    └── subdir/
        └── file3.txt

    After renaming

    s3://bucket-name/<new_folder_prefix>/
    ├── file1.txt
    ├── file2.txt
    └── subdir/
        └── file3.txt

    :param bucket_name: Name of the S3 bucket
    :param source_folder: Source "folder" path in the bucket
    :param destination_folder: Destination "folder" path in the bucket
    """
    # Copy the object to new location and delete the old location
    old_files = list_all_objects_in_bucket_by_prefix(bucket_name=bucket_name, prefix=old_folder_prefix)
    for file in old_files:
        try:
            target_key = file.replace(old_folder_prefix, new_folder_prefix)
            move_object_in_bucket(bucket_name=bucket_name, source_key=file, destination_key=target_key)
        except ClientError as err:
            print(f"Error renaming {file} to {target_key}: {err}")
            continue


def move_object_in_bucket(bucket_name: str, source_key: str, destination_key: str):
    """
    Move an object within an S3 bucket by copying to the new key and deleting the old key.

    :param bucket_name: Name of the S3 bucket
    :param source_key: Source key of the object to move
    :param destination_key: Destination key of the object
    """
    try:
        S3_CLIENT.copy(
            CopySource={"Bucket": bucket_name, "Key": source_key},
            Bucket=bucket_name,
            Key=destination_key,
        )
        print(f"Copied {source_key} to {destination_key}")

        S3_CLIENT.delete_object(Bucket=bucket_name, Key=source_key)
        print(f"Deleted {source_key}")
    except ClientError as err:
        print(f"Error moving {source_key} to {destination_key}: {err}")
        raise

In [125]:
"""Test the rename_folder_in_bucket function."""

# def upload_file_to_bucket(
#     local_fpath: str | Path,
#     bucket_name: str,
#     target_key_in_bucket: str,
# ):
#     """
#     Upload a file to an S3 bucket.

#     :param local_fpath: Local file path to upload
#     :param bucket_name: Name of the bucket to upload the file to
#     :param target_key_in_bucket: Key to upload the file to in the bucket
#     """
#     local_fpath = Path(local_fpath)
#     with open(local_fpath, "rb") as file:
#         S3_CLIENT.put_object(Bucket=bucket_name, Key=target_key_in_bucket, Body=file)


# # Create test objects in the source folder
test_source_folder = "nested/source-folder/"
test_dest_folder = "nested/destination-folder/"

# create_bucket(bucket_name=BUCKET_NAME)

# # Upload test files to source folder
# upload_file_to_bucket(
#     local_fpath="test_dir/file1.txt", bucket_name=BUCKET_NAME, target_key_in_bucket=test_source_folder + "file1.txt"
# )
# upload_file_to_bucket(
#     local_fpath="test_dir/file2.txt", bucket_name=BUCKET_NAME, target_key_in_bucket=test_source_folder + "file2.txt"
# )
# upload_file_to_bucket(
#     local_fpath="test_dir/subdir/file3.txt",
#     bucket_name=BUCKET_NAME,
#     target_key_in_bucket=test_source_folder + "subdir/file3.txt",
# )

recursively_upload_dir_to_bucket(
    bucket_name=BUCKET_NAME,
    local_dir_fpath=test_dir,
    target_root_prefix_in_bucket=test_source_folder,
)

# Rename the source folder to the destination folder
rename_folder_in_bucket(
    bucket_name=BUCKET_NAME,
    old_folder_prefix=test_source_folder,
    new_folder_prefix=test_dest_folder,
)

# List all objects in the destination folder
objects = list_all_objects_in_bucket_by_prefix(bucket_name=BUCKET_NAME, prefix=test_dest_folder)
print(objects)

Writing object to path 's3://cloud-course-bucket-1a4e/nested/source-folder/file1.txt'

Writing object to path 's3://cloud-course-bucket-1a4e/nested/source-folder/file2.txt'

Writing object to path 's3://cloud-course-bucket-1a4e/nested/source-folder/subdir/file3.txt'

Directory 'test_dir' uploaded successfully to S3 bucket 'cloud-course-bucket-1a4e' with prefix 
'nested/source-folder/'.

Copied nested/source-folder/file1.txt to nested/destination-folder/file1.txt

...

Copied nested/source-folder/file2.txt to nested/destination-folder/file2.txt

Deleted nested/source-folder/file2.txt

Copied nested/source-folder/subdir/file3.txt to nested/destination-folder/subdir/file3.txt

Deleted nested/source-folder/subdir/file3.txt

[
    'nested/destination-folder/file1.txt',
    'nested/destination-folder/file2.txt',
    'nested/destination-folder/subdir/file3.txt'
]

In [126]:
delete_bucket(BUCKET_NAME)

Deleted objects in bucket 'cloud-course-bucket-1a4e'. Proceeding to delete the bucket.

Bucket 'cloud-course-bucket-1a4e' deleted successfully!

{'ResponseMetadata': {'RequestId': 'NJK18W8VD7329525',
  'HostId': 'b3afPP2Q3Galbr6GIm1EwZl5lY72xLDJGxnNaIXWqrOtnhdGzidsPGIoOBqEjmG4032NwGIcpoY=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'b3afPP2Q3Galbr6GIm1EwZl5lY72xLDJGxnNaIXWqrOtnhdGzidsPGIoOBqEjmG4032NwGIcpoY=',
   'x-amz-request-id': 'NJK18W8VD7329525',
   'date': 'Fri, 07 Jun 2024 12:50:38 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

In [127]:
# delete_bucket("python-cloud-eng-course-bucket-avr")

Deleted objects in bucket 'python-cloud-eng-course-bucket-avr'. Proceeding to delete the bucket.

Bucket 'python-cloud-eng-course-bucket-avr' deleted successfully!

{'ResponseMetadata': {'RequestId': '3E1KXZ266S3W8X6F',
  'HostId': 'CGj6LbGfR14L4b9uahCIE/9SiXZnokfTqaj7vcS8zA/PEsm9d2iv3+PdaBjRxzu+DT8NiH/O39o=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'CGj6LbGfR14L4b9uahCIE/9SiXZnokfTqaj7vcS8zA/PEsm9d2iv3+PdaBjRxzu+DT8NiH/O39o=',
   'x-amz-request-id': '3E1KXZ266S3W8X6F',
   'date': 'Fri, 07 Jun 2024 12:50:56 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}